In [4]:
# try resampling first then combining orders
#import import_ipynb
#from rescale import get_rescaled_wave_soln
#import numpy as np

importing Jupyter notebook from rescale.ipynb


In [11]:
def resample_order(wave_data_new, wave_data_old, data):
    # MODIFED VERSION!!
    import numpy as np
    
    """
    Resample a spectrum onto a new wavelength solution
    by averaging intensity values within +/- 0.017 Å on the new wavelength array
    (typically an average of two intesity values for each wavelength,
    with exception to regions where spectral orders overlap in which case
    there is an average of more intensity values per wavelength)
    Args:
        wave_data_new (numpy.ndarray): Wavelength array used
        to resample the data
        wave_data_old (numpy.ndarray): Initial wavelength array corresponding
        to the data
        data (numpy.ndarray): Initial intensity data
    Returns:
        data_new (numpy.ndarray): New intensity data that corresponds to the
        new wavelength solution
    """

    # Focus data onto the wavelength region 4997-6338 Å
    # index 3055 in order 31 roughly corresponds to 5000 Å
    # index 3642 in order 52 roughly corresponds to 6338 Å
    # note: here 30 actually corresponds to spectral order 31,
    # due to python starting indexing at 0,
    # so for reference 0 is order 1, and 1 is order 2, etc.

    # w_lst = wavelength list
    # s_lst = intensity list

   # w_lst = []
   # s_lst = []
   # w_lst.append(wave_data_old[30, 3055:4607].tolist())
   # s_lst.append(data[30, 3055:4607].tolist())
   # for i in range(20):
   #     w_lst.append(wave_data_old[31 + i, :4607].tolist())
   #     s_lst.append(data[31 + i, :4607].tolist())
   # w_lst.append(wave_data_old[51, :3642].tolist())
   # s_lst.append(data[51, :3642].tolist())
   # w_lst = [item for sublist in w_lst for item in sublist]
   # s_lst = [item for sublist in s_lst for item in sublist]
    
    w_lst = wave_data_old
    s_lst = data

    # create array that has wavelengths in column 0 and intensity in column 1
    ws = np.zeros((len(w_lst), 2))
    for i in range(len(w_lst)):
        ws[i,0] = w_lst[i]
        ws[i,1] = s_lst[i]

    # sort the array by increasing wavelength
    # the result of merging spectral orders does not strictly increase
    # because orders overlap in wavelength ranges
    # ws = ws[ws[:, 0].argsort()]

    # select to skip first 137 values (closer to 4997 Å) - via eye test
    # ws = ws[137:]
    
    # average intensity values within +/- 0.017 Å on the new wavelength scale
    
    # comparisons between wavelengths in the old and new scale start from the
    # beginning of the old wavelength array for each new wavelength value
    # inefficient, so data points in the front of the array are clipped every
    # 1000 comparisons to reduce time wasted on wavelength comparisons
    data_new = []
    count = 0
    pairs = ws.tolist() # pairs -> (old wavelength, old intensity)
    for w in wave_data_new: # for every new wavelength value
        if (count != 0):
            if ((count % 1000 == 0)): # for every thousand comparisons

                # remove 800 pairs from the beginning of the
                # list of old values
                removed_1 = pairs[:800]
                removed_1.reverse()
                del pairs[:800]

                # if we remove too many,
                # the new wavelength value is less than the
                # starting wavelength value in the old list
                # i.e. (w_new - w_old < 0)
                # so we return some of the pairs
                if (w - pairs[0][0] < 0):
                    for pair in removed_1:
                        pairs.insert(0, pair)
                        break # currently breaks after inserting one pair

                # while the new wavelength value is greater than
                # the starting wavelength in the old list by more than a tenth,
                # remove a pair
                # (to improve efficiency, avoiding unneccessary comparisons)
                while ((w - pairs[0][0] > .1)):
                    removed_2 = pairs[0]
                    del pairs[0]
                    if (w - pairs[0][0] < 0): # replace if (w_new - w_old < 0)
                        pairs.insert(0, removed_2)
                        break
        avg_lst = []
        some_found = False
        for pair in pairs:
            # comparison process between new and old wavelength values
            if (abs(w - pair[0]) < .017):
                avg_lst.append(pair[1])
                some_found = True
            elif (some_found): break # break when the inequality no longer holds
        data_new.append(np.mean(np.asarray(avg_lst)))
        count += 1

        
    print('Finished resample') #ADZ DELETE THIS 6/23/20
    return data_new

In [5]:
# TO DO
# for each order 
# 1) find the section of the rescaled solution that you want to sample onto (ie. maybe +/- 0.17 on either side of the first and last new wl value that fit in that order)
# 2) pass the order data and the section of the rescaled solution into resample
#    --> modify resample so that it only deals with one order
# 3) For overlap region, just average the two values.

In [ ]:
#lib_wl_soln = get_rescaled_wave_soln()

In [ ]:
#for order in np.arange(30,52,1):
#    order_values = spectrum[order][100:-100] # truncate the ends becuase the deblazing doesn't work well here --> unless in future iteration do this in deblazing
#    apf_wl_values = apf_wave_soln[order][100:-100]
#    first = apf_wl_values[order][0]
#   last = apf_wl_values[order][-1]
#    new_first = first - 0.017
#    new_last = last + 0.17
#    new_wl_section = lib_wl_soln[(new_first <= lib_wl_soln) * (new_last >= lib_wl_soln)]
#    resampled_order = resample_order(new_wl_section, apf_wl_values, order_values)
    